In [17]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
df = pd.read_csv("installments_payments.csv")

# 데이터 타입 변경으로 메모리 사용량 줄이기

In [3]:
df.shape

(13605401, 8)

In [4]:
df.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13605401 entries, 0 to 13605400
Data columns (total 8 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_PREV              int64  
 1   SK_ID_CURR              int64  
 2   NUM_INSTALMENT_VERSION  float64
 3   NUM_INSTALMENT_NUMBER   int64  
 4   DAYS_INSTALMENT         float64
 5   DAYS_ENTRY_PAYMENT      float64
 6   AMT_INSTALMENT          float64
 7   AMT_PAYMENT             float64
dtypes: float64(5), int64(3)
memory usage: 830.4 MB


* 메모리 사용을 830메가 하는 중
* Data Type도 불필요하게 과함

In [6]:
for col in df.columns:
    print("컬럼명 :", col, "최소값 :", df[col].min(), "최대값 :", df[col].max())

컬럼명 : SK_ID_PREV 최소값 : 1000001 최대값 : 2843499
컬럼명 : SK_ID_CURR 최소값 : 100001 최대값 : 456255
컬럼명 : NUM_INSTALMENT_VERSION 최소값 : 0.0 최대값 : 178.0
컬럼명 : NUM_INSTALMENT_NUMBER 최소값 : 1 최대값 : 277
컬럼명 : DAYS_INSTALMENT 최소값 : -2922.0 최대값 : -1.0
컬럼명 : DAYS_ENTRY_PAYMENT 최소값 : -4921.0 최대값 : -1.0
컬럼명 : AMT_INSTALMENT 최소값 : 0.0 최대값 : 3771487.845
컬럼명 : AMT_PAYMENT 최소값 : 0.0 최대값 : 3771487.845


In [7]:
# 각 데이터에 맞는 데이터 Type 변경
df = df.astype({
    "SK_ID_PREV": np.int32,
    "SK_ID_CURR": np.int32,
    "NUM_INSTALMENT_VERSION": np.float16,
    "NUM_INSTALMENT_NUMBER": np.int16,
    "DAYS_INSTALMENT": np.float16,
    "DAYS_ENTRY_PAYMENT": np.float16
})

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13605401 entries, 0 to 13605400
Data columns (total 8 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_PREV              int32  
 1   SK_ID_CURR              int32  
 2   NUM_INSTALMENT_VERSION  float16
 3   NUM_INSTALMENT_NUMBER   int16  
 4   DAYS_INSTALMENT         float16
 5   DAYS_ENTRY_PAYMENT      float16
 6   AMT_INSTALMENT          float64
 7   AMT_PAYMENT             float64
dtypes: float16(3), float64(2), int16(1), int32(2)
memory usage: 415.2 MB


* 메모리 사용량이 415메가로 절반으로 줄어듬

# 데이터 처리 속도 증가시키기

### iloc로 접근하는 경우
* range를 이용해 for문 실행
* iloc, loc와 같은 인덱스 기반 접근
* 결과 : 34분 소요 (속도 : 6700 it/s)

In [19]:
%%time
cnt = 0
for i in tqdm(range(len(df)), total = len(df), position=0, leave=True):
    d = df.iloc[i, :]
    cnt +=1

100%|████████████████████████████████████████████████████████████████████| 13605401/13605401 [34:33<00:00, 6562.42it/s]

CPU times: user 34min 13s, sys: 24.7 s, total: 34min 37s
Wall time: 34min 33s


In [21]:
for i in tqdm(range(len(df)), total = len(df), position=0, leave=True):
    print(type(i), i)
    print(df.iloc[i, :])
    break

  0%|                                                                                     | 0/13605401 [00:00<?, ?it/s]

<class 'int'> 0
SK_ID_PREV                1054186.00
SK_ID_CURR                 161674.00
NUM_INSTALMENT_VERSION          1.00
NUM_INSTALMENT_NUMBER           6.00
DAYS_INSTALMENT             -1180.00
DAYS_ENTRY_PAYMENT          -1187.00
AMT_INSTALMENT               6948.36
AMT_PAYMENT                  6948.36
Name: 0, dtype: float64


### dataframe의 iterrows() 사용하기
* 결과 : 10분 소요 (속도 : 20000 it/s)

In [22]:
%%time
cnt = 0
for rows in tqdm(df.iterrows(), total=len(df), position=0, leave=True):
    d = rows[1].values
    cnt += 1

100%|███████████████████████████████████████████████████████████████████| 13605401/13605401 [10:51<00:00, 20892.61it/s]

CPU times: user 10min 45s, sys: 7.55 s, total: 10min 53s
Wall time: 10min 51s


In [24]:
for rows in tqdm(df.iterrows(), total=len(df), position=0, leave=True):
    print(type(rows), rows)
    print(rows[1])
    d = rows[1].values
    cnt += 1
    break

  0%|                                                                                     | 0/13605401 [00:00<?, ?it/s]

<class 'tuple'> (0, SK_ID_PREV                1054186.00
SK_ID_CURR                 161674.00
NUM_INSTALMENT_VERSION          1.00
NUM_INSTALMENT_NUMBER           6.00
DAYS_INSTALMENT             -1180.00
DAYS_ENTRY_PAYMENT          -1187.00
AMT_INSTALMENT               6948.36
AMT_PAYMENT                  6948.36
Name: 0, dtype: float64)
SK_ID_PREV                1054186.00
SK_ID_CURR                 161674.00
NUM_INSTALMENT_VERSION          1.00
NUM_INSTALMENT_NUMBER           6.00
DAYS_INSTALMENT             -1180.00
DAYS_ENTRY_PAYMENT          -1187.00
AMT_INSTALMENT               6948.36
AMT_PAYMENT                  6948.36
Name: 0, dtype: float64


### numpy로 접근하기
* df.to_numpy 이용
* 결과 : 6초 소요 (속도 : 2300000 it/s)

In [25]:
%%time
cnt = 0
for rows in tqdm(df.to_numpy(), total=len(df), position=0, leave=True):
    d = rows
    cnt += 1

100%|█████████████████████████████████████████████████████████████████| 13605401/13605401 [00:05<00:00, 2292756.17it/s]

CPU times: user 6.14 s, sys: 334 ms, total: 6.47 s
Wall time: 6.45 s


# 데이터 저장 시 용량 감소시키기

### npz로 저장하기 
* npz로 저장하면 key-value 형태로 저장할 수 있다. key는 컬럼, value는 그에 대한 값이 저장될 수 있다
* 기존 : 723 MB, 변경 : 435 MB

In [27]:
SK_ID_PREV = df["SK_ID_PREV"].values
SK_ID_CURR = df["SK_ID_CURR"].values
NUM_INSTALMENT_VERSION = df["NUM_INSTALMENT_VERSION"].values
NUM_INSTALMENT_NUMBER = df["NUM_INSTALMENT_NUMBER"].values
DAYS_INSTALMENT = df["DAYS_INSTALMENT"].values
DAYS_ENTRY_PAYMENT = df["DAYS_ENTRY_PAYMENT"].values
AMT_INSTALMENT = df["AMT_INSTALMENT"].values
AMT_PAYMENT = df["AMT_PAYMENT"].values

In [28]:
np.savez('data.npz', SK_ID_PREV=SK_ID_PREV, SK_ID_CURR=SK_ID_CURR, NUM_INSTALMENT_VERSION=NUM_INSTALMENT_VERSION, NUM_INSTALMENT_NUMBER=NUM_INSTALMENT_NUMBER, DAYS_INSTALMENT=DAYS_INSTALMENT, DAYS_ENTRY_PAYMENT=DAYS_ENTRY_PAYMENT, AMT_INSTALMENT=AMT_INSTALMENT, AMT_PAYMENT=AMT_PAYMENT)

### pickle로 저장하기
* 기존 : 723 MB, 변경 : 435 MB

In [29]:
import pickle

with open("data.pickle", "wb") as f:
    pickle.dump(df, f, protocol=pickle.HIGHEST_PROTOCOL)